# OpenVINO APIの基礎を学ぶ
ここではシンプルな画像分類(Classification)プログラムを実行しながらOpenVINO Python APIの基礎を学びます。

### 入力画像とラベルデータの準備
まずは推論に使用する入力画像ファイルと、クラスラベルのテキストファイルをOpenVINOのdemoディレクトリからコピーしてきます。

In [ ]:
# Linux
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

In [ ]:
# Windows
!echo %INTEL_OPENVINO_DIR%
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car.png" .
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\squeezenet1.1.labels" synset_words.txt

コピーしてきた推論入力の絵を表示して確認します。

In [ ]:
from IPython.display import Image
Image('car.png')

### 推論に使用するIRモデルデータの準備
推論に使用するモデルを`Model downloader`でダウンロードし、`Model converter`でIRモデルに変換します。

In [ ]:
# Linux  
!python3 $INTEL_OPENVINO_DIR/deployment_tools/open_model_zoo/tools/downloader/downloader.py --name googlenet-v3
!python3 $INTEL_OPENVINO_DIR/deployment_tools/open_model_zoo/tools/downloader/converter.py  --name googlenet-v3 --precisions FP16
!ls public/googlenet-v3/FP16 -l

In [ ]:
# Windows  
!python "%INTEL_OPENVINO_DIR%\deployment_tools\open_model_zoo\tools\downloader\downloader.py" --name googlenet-v3 
!python "%INTEL_OPENVINO_DIR%\deployment_tools\open_model_zoo\tools\downloader\converter.py"  --name googlenet-v3 --precisions FP16
!dir public\googlenet-v3\FP16

----
ここからPythonプログラム本体となります。  

### プログラムで使用するモジュールをインポートする

In [ ]:
import cv2
import numpy as np
from openvino.inference_engine import IECore

### クラスラベルテキストデータを読み込む

In [ ]:
label = [ s.replace('\n', '') for s in open('synset_words.txt').readlines() ]

print(len(label), 'labels read')   # 読み込んだラベルの個数を表示
print(label[:20])                  # ラベルの先頭の20個を表示

### Inference Engineオブジェクトを生成する
- Inference Engineコアオブジェクトを生成
- モデルデータの読み込み
- 入出力ブロブ(バッファ)の情報取得  

もちろんモデルの入出力ブロブ名やシェイプが既知の場合、わざわざモデルデータから取得せずにハードコードしてしまっても構いません。

In [ ]:
# Inference Engineコアオブジェクトの生成
ie = IECore()

# IRモデルファイルの読み込み
model = './public/googlenet-v3/FP16/googlenet-v3'
net = ie.read_network(model=model+'.xml', weights=model+'.bin')

# 入出力blobの名前の取得、入力blobのシェイプの取得
input_blob_name  = list(net.input_info.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.input_info[input_blob_name].tensor_desc.dims

# モデルの情報の表示
print(input_blob_name, batch, channel, height, width)
print(output_blob_name, net.outputs[output_blob_name].shape)

### モデルをIE coreオブジェクトにロードする
読み込んだネットワークオブジェクトをInference engineプラグインにセットします。  
ここでは推論実行デバイスとして`CPU`を指定していますが、`'GPU'`, `'MYRIAD'`, `'HETERO:FPGA,CPU'`などを指定することでほかのデバイスを指定することも可能です。  
OpenVINOを使ったアプリケーションではほとんどの場合この部分を書き換えるだけで簡単に推論実行デバイスを切り替えることが可能です。  

**Note**: DevCloudの開発サーバーはCPUしか持っていません。その他のデバイスを試す場合は、DevCloudのedge computing nodeを使用する必要があります。

In [ ]:
exec_net = ie.load_network(network=net, device_name='CPU', num_requests=1)

### 推論入力データを準備する
推論入力画像を読み込み、入力ブロブのシェイプに合わせて変形します。

In [ ]:
print('input blob: name="{}", N={}, C={}, H={}, W={}'.format(input_blob_name, batch, channel, height, width))
img = cv2.imread('car.png')
img = cv2.resize(img, (width,height))
img = img.transpose((2, 0, 1))
img = img.reshape((1, channel, height, width))

print(img.shape)

### 推論を実行する  
`infer()` APIはブロッキング関数です。推論が終了すると制御が戻り、次の行を実行します。  
入力は`{入力blob名:入力データ}`の辞書の形で渡します。

In [ ]:
res = exec_net.infer(inputs={input_blob_name: img})

print(res[output_blob_name].shape)

### 推論結果を表示する 
今回使用しているgooglenet-v3モデルは1000のクラスを持つImageNetデータセットで学習しています。1000のクラスごとの確率をFP32の数値で出力します。つまり推論結果は1000の要素を持つFloatの配列です。  
推論結果をソートし、確率の高い順に5つ候補を表示させています。

In [ ]:
result=res[output_blob_name][0]
idx = np.argsort(result)[::-1]
for i in range(5):
    print(idx[i], result[idx[i]], label[idx[i]-1])

----
ここまでで一番シンプルな画像分類のプログラムの構成について学びました。  
OpenVINO APIはシンプルに作られているので20行程度のPythonプログラムでディープラーニングを使った画像分類が可能です。  
入力画像をいろいろ変えてみたりして実験してみましょう。

## Next => OpenVINOを使った物体検出プログラムの基礎 - [object-detection-ssd.ipynb](./object-detection-ssd.ipynb)